In [11]:
import pickle
import nltk
import pandas as pd
import csv

from tqdm import tqdm
from googletrans import Translator

translator = Translator()

This notebook contains the procedures necessary to translate the datasets into other languages. The translation results are already included in the repository

In [2]:
def read_data():
    columns = ['context_id','context_2','context_1','context_0','reply_id','reply','label','confidence']

    test_df = pd.read_csv("./data/final.tsv", sep="\t", header=None, quoting=csv.QUOTE_NONE)
    train_df = pd.read_csv("./data/train.tsv", sep="\t", header=None, quoting=csv.QUOTE_NONE)

    train_df = train_df.fillna("")
    test_df = test_df.fillna("")

    test_df.columns = columns[:-2]
    train_df.columns = columns
    
    return train_df, test_df

In [3]:
train_df, test_df = read_data()

In [5]:
R = {}

alltexts = set()

for df in [train_df, test_df]:
    for cl in ['context_2','context_1','context_0','reply']:
        alltexts.update(df[cl].tolist())
        
alltexts = list(alltexts)
print(len(alltexts))

184721


In [6]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [ ]:
for b in tqdm(batch(alltexts, 8), total=int(len(alltexts)/8)):
    
    b = [s for s in b if s not in R]
    if len(b):
        translations = translator.translate(b, dest='en')
        for t, s in zip(translations, b):
            R[s] = " ".join(nltk.word_tokenize(t.text)).lower()

In [62]:
pickle.dump(R, open("./assets/translations/english_preprocessed.pkl","wb"))